El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Instrucciones del proyecto
1. Descarga y examina los datos.
2. Entrena diferentes modelos con varios hiperparámetros (debes hacer al menos dos modelos diferentes, pero más es mejor. Recuerda, varias implementaciones de potenciación del gradiente no cuentan como modelos diferentes). El punto principal de este paso es comparar métodos de potenciación del gradiente con bosque aleatorio, árbol de decisión y regresión lineal.
3. Analiza la velocidad y la calidad de los modelos.

## Observaciones:

- Utiliza la métrica RECM para evaluar los modelos.
- La regresión lineal no es muy buena para el ajuste de hiperparámetros, pero es perfecta para hacer una prueba de cordura de otros métodos. Si la potenciación del gradiente funciona peor que la regresión lineal, definitivamente algo salió mal.
- Aprende por tu propia cuenta sobre la librería LightGBM y sus herramientas para crear modelos de potenciación del gradiente (gradient boosting).
Idealmente, tu proyecto debe tener regresión lineal para una prueba de cordura, un algoritmo basado en árbol con ajuste de hiperparámetros (preferiblemente, bosque aleatorio), LightGBM con ajuste de hiperparámetros (prueba un par de conjuntos), y CatBoost y XGBoost con ajuste de hiperparámetros (opcional).
- Toma nota de la codificación de características categóricas para algoritmos simples. LightGBM y CatBoost tienen su implementación, pero XGBoost requiere OHE.
- Puedes usar un comando especial para encontrar el tiempo de ejecución del código de celda en Jupyter Notebook. Encuentra ese comando.
- Dado que el entrenamiento de un modelo de potenciación del gradiente puede llevar mucho tiempo, cambia solo algunos parámetros del modelo.
- Si Jupyter Notebook deja de funcionar, elimina las variables excesivas por medio del operador del:
```
del features_train
  
```

## Descripción de los datos
El dataset está almacenado en el archivo /datasets/car_data.csv. descargar dataset.

Características

- DateCrawled — fecha en la que se descargó el perfil de la base de datos
- VehicleType — tipo de carrocería del vehículo
- RegistrationYear — año de matriculación del vehículo
- Gearbox — tipo de caja de cambios
- Power — potencia (CV)
- Model — modelo del vehículo
- Mileage — kilometraje (medido en km de acuerdo con las especificidades regionales del conjunto de datos)
- RegistrationMonth — mes de matriculación del vehículo
- FuelType — tipo de combustible
- Brand — marca del vehículo
- NotRepaired — vehículo con o sin reparación
- DateCreated — fecha de creación del perfil
- NumberOfPictures — número de fotos del vehículo
- PostalCode — código postal del propietario del perfil (usuario)
- LastSeen — fecha de la última vez que el usuario estuvo activo

Objetivo

- Price — precio (en euros)

## Evaluación del proyecto
Hemos definido los criterios de evaluación para el proyecto. Léelos con atención antes de pasar al ejercicio.

Esto es en lo que se fijarán los revisores al examinar tu proyecto:

- ¿Seguiste todos los pasos de las instrucciones?
- ¿Cómo preparaste los datos?
- ¿Qué modelos e hiperparámetros consideraste?
- ¿Conseguiste evitar la duplicación del código?
- ¿Cuáles son tus hallazgos?
- ¿Mantuviste la estructura del proyecto?
- ¿Mantuviste el código ordenado?

In [1]:
! pip install lightgbm
! pip install catboost
! pip install xgboost

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer

from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

## Preparación de datos

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/Davichobacter/data_science_tt/refs/heads/main/Sprint_15/datasets/car_data.csv')
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17


In [4]:
def explorar_dataset(df):
    """
    Explora un DataFrame mostrando información clave.

    Esta función imprime: información general del DataFrame (df.info()),
    sus dimensiones (df.shape), las primeras 15 filas (df.head(15)),
    estadísticas descriptivas (df.describe()), el conteo de valores nulos
    (df.isnull().sum()) y el conteo de filas duplicadas (df.duplicated().sum()).

    Parámetros:
        df (pd.DataFrame): El DataFrame a explorar.
    """
    print('---' * 10, '\n', f'Información del dataframe')
    print(df.info())
    print('---' * 10, '\n', f'Dimensiones del dataframe')
    print(df.shape)
    print('---' * 10, '\n', f'Primeras filas del dataframe')
    print(df.head(15))
    print('---' * 10, '\n', f'Descripción del dataframe')
    print(df.describe())
    print('---' * 10, '\n', f'Valores nulos del dataframe')
    print(df.isnull().sum())
    print('---' * 10, '\n', f'Valores duplicados del dataframe')
    print(df.duplicated().sum())
    print('---' * 10)

In [5]:
explorar_dataset(df)

------------------------------ 
 Información del dataframe
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen 

In [6]:
def proporcion_nulos(df):
    """
    Calcula la proporción de valores nulos en una columna específica de un DataFrame.

    Parámetros:
        df (pd.DataFrame): El DataFrame al que pertenece la columna.

    Retorna:
        pd.Series: Una serie que contiene las proporciones de valores nulos en la columna.
    """

    return df.isna().mean().sort_values(ascending=False) * 100

In [7]:
proporcion_nulos(df)

,0
NotRepaired,20.079070
VehicleType,10.579368
FuelType,9.282697
Gearbox,5.596709
Model,5.560588
Price,0.000000
RegistrationYear,0.000000
DateCrawled,0.000000
Mileage,0.000000
Power,0.000000


In [8]:
df['NotRepaired'].value_counts()

,count
NotRepaired,
no,247161
yes,36054


La columna `NotRepaired` tiene aproximadamente un 20% de valores nulos. Para manejar estos valores, podemos considerar las siguientes estrategias:

1.  **Imputación con un valor por defecto (por ejemplo, 'unknown' o 'not_specified')**: Esta es una opción segura, ya que crea una nueva categoría para los valores faltantes sin hacer suposiciones sobre si el coche fue reparado o no. Esto es útil si la ausencia de información sobre reparación podría ser una categoría significativa en sí misma.

    ```python
    df['NotRepaired'] = df['NotRepaired'].fillna('unknown')
    ```

2.  **Imputación con la moda (valor más frecuente)**: Si asumimos que la mayoría de los coches sin información no han sido reparados (o el caso más común), podríamos rellenar los nulos con el valor predominante ('no').

    ```python
    # Obtener el valor más frecuente
    most_frequent_value = df['NotRepaired'].mode()[0]
    df['NotRepaired'] = df['NotRepaired'].fillna(most_frequent_value)
    ```

3.  **Eliminar filas**: Dado que el 20% es una proporción considerable, eliminar las filas con valores nulos en esta columna podría resultar en una pérdida significativa de datos y no es recomendable en este caso.

4.  **Eliminar la columna**: La información de si un coche ha sido reparado o no es probablemente un factor importante en su precio, por lo que eliminar esta columna no sería ideal.

**Recomendación**: La imputación con una nueva categoría como 'unknown' o 'not_specified' es generalmente la mejor práctica para columnas categóricas con un porcentaje significativo de valores nulos, ya que preserva la información existente y no introduce sesgos.

Además, antes de usar esta columna en modelos que no manejan categorías directamente (como regresión lineal o algunos modelos de árbol sin su propia codificación), será necesario aplicar alguna técnica de codificación categórica como One-Hot Encoding o Label Encoding.

In [9]:
df['NotRepaired'] = df['NotRepaired'].fillna('unknown')

In [10]:
proporcion_nulos(df)

,0
VehicleType,10.579368
FuelType,9.282697
Gearbox,5.596709
Model,5.560588
RegistrationYear,0.000000
Price,0.000000
Power,0.000000
DateCrawled,0.000000
Mileage,0.000000
RegistrationMonth,0.000000


Después de haber imputado los valores nulos de la columna `NotRepaired`, aún quedan algunas columnas con valores faltantes:

- `VehicleType`: ~10.58% de nulos
- `FuelType`: ~9.28% de nulos
- `Gearbox`: ~5.60% de nulos
- `Model`: ~5.56% de nulos

Todas estas columnas son de tipo categórico. Las estrategias más adecuadas para manejar estos valores nulos son:

1.  **Imputación con una nueva categoría ('unknown' o 'not_specified')**: Esta es la opción más robusta y generalmente recomendada para columnas categóricas con valores faltantes. Al asignar una categoría como 'unknown', no estamos haciendo suposiciones sobre el valor real y permitimos que el modelo aprenda si la ausencia de información es significativa por sí misma. Esto es especialmente útil si la proporción de nulos es considerable, como en el caso de `VehicleType` y `FuelType`.

    ```python
    df['VehicleType'] = df['VehicleType'].fillna('unknown')
    df['FuelType'] = df['FuelType'].fillna('unknown')
    df['Gearbox'] = df['Gearbox'].fillna('unknown')
    df['Model'] = df['Model'].fillna('unknown')
    ```

2.  **Imputación con la moda (valor más frecuente)**: Si se cree que el valor faltante probablemente corresponde a la categoría más común, se podría imputar con la moda de la columna. Sin embargo, esto puede introducir un sesgo si la ausencia de datos no es aleatoria. Es una opción más agresiva que la imputación con 'unknown'.

    ```python
    # Ejemplo para una columna:
    # most_frequent_vehicle_type = df['VehicleType'].mode()[0]
    # df['VehicleType'] = df['VehicleType'].fillna(most_frequent_vehicle_type)
    ```

3.  **Eliminar filas**: Dada la proporción de nulos (hasta ~10.58%), eliminar las filas con valores faltantes en estas columnas podría llevar a una pérdida significativa de datos. Por ejemplo, eliminar filas con nulos en `VehicleType` eliminaría más del 10% del dataset, lo cual no es recomendable a menos que el volumen de datos sea muy grande y la pérdida no afecte la representatividad.

**Recomendación**: Para estas columnas categóricas, la estrategia de **imputar con una nueva categoría 'unknown'** es la más segura y recomendada. Preserva toda la información disponible y permite que los modelos capturen el significado de los valores ausentes.

Después de la imputación, será necesario aplicar técnicas de codificación para las variables categóricas (como One-Hot Encoding o Label Encoding) antes de entrenar la mayoría de los modelos de machine learning (especialmente los basados en regresión lineal o árboles que no manejan categorías nativamente).

In [11]:
df['VehicleType'].value_counts()

,count
VehicleType,
sedan,91457
small,79831
wagon,65166
bus,28775
convertible,20203
coupe,16163
suv,11996
other,3288


In [12]:
df['VehicleType'] = df['VehicleType'].fillna('other')

In [13]:
df['FuelType'].value_counts()

,count
FuelType,
petrol,216352
gasoline,98720
lpg,5310
cng,565
hybrid,233
other,204
electric,90


In [14]:
df['FuelType'] = df['FuelType'].fillna('other')

In [15]:
df['Gearbox'].value_counts()

,count
Gearbox,
manual,268251
auto,66285


In [16]:
df['Model'].value_counts()

,count
Model,
golf,29232
other,24421
3er,19761
polo,13066
corsa,12570
...,...
i3,8
serie_3,4
rangerover,4


In [17]:
df = df.dropna()

In [18]:
proporcion_nulos(df)

,0
DateCrawled,0.0
Price,0.0
VehicleType,0.0
RegistrationYear,0.0
Gearbox,0.0
Power,0.0
Model,0.0
Mileage,0.0
RegistrationMonth,0.0
FuelType,0.0


El camino seguido para manejar los valores nulos en las columnas fue el siguiente:

1.  **Columna `NotRepaired` (20.08% de nulos):**
    *   Esta columna categórica fue imputada con el valor `'unknown'`. Se eligió esta estrategia para no hacer suposiciones sobre si el vehículo había sido reparado o no, permitiendo que la ausencia de información se tratara como una categoría propia.

2.  **Columnas `VehicleType` (10.58% de nulos) y `FuelType` (9.28% de nulos):**
    *   Estas columnas categóricas se imputaron con el valor `'other'`. Aunque inicialmente se consideró `'unknown'`, al revisar los conteos de valores únicos, se observó que la categoría `'other'` ya existía en ambas columnas, lo que sugiere que podría ser una categoría adecuada para valores no especificados o de baja frecuencia. Esta imputación ayuda a mantener la coherencia con las categorías existentes.

3.  **Columnas `Gearbox` (5.60% de nulos) y `Model` (5.56% de nulos):**
    *   En lugar de imputar estos valores, se optó por eliminar las filas que contenían nulos en estas columnas mediante el método `df.dropna()`. Aunque la proporción de nulos no es insignificante, se decidió que para estas columnas, la eliminación de las filas era una alternativa viable para simplificar el conjunto de datos y asegurar la calidad de los registros restantes, asumiendo que el impacto en la cantidad total de datos no sería excesivamente perjudicial para el entrenamiento de los modelos.

In [19]:
df['NumberOfPictures'].value_counts()

,count
NumberOfPictures,
0,318962


In [20]:
df = df.drop(['NumberOfPictures', 'PostalCode'], axis=1)

Las columnas `NumberOfPictures` y `PostalCode` fueron eliminadas del conjunto de datos por las siguientes razones:

-   **`NumberOfPictures`**: Esta columna contenía un valor constante de `0` para todas las entradas. Una columna con valores idénticos para todas las filas no aporta información útil para el entrenamiento de un modelo de machine learning, ya que no presenta variabilidad que el modelo pueda aprender. Por lo tanto, su eliminación ayuda a reducir la dimensionalidad del dataset sin perder información relevante.

-   **`PostalCode`**: Si bien los códigos postales pueden contener información geográfica, para este problema de predicción de precios de automóviles usados, la granularidad y la naturaleza categórica de `PostalCode` podrían no ser directamente útiles sin un procesamiento adicional complejo (como la conversión a coordenadas geográficas o la agregación por regiones). Además, su alta cardinalidad podría introducir ruido o requerir una codificación que aumente significativamente la dimensionalidad. Dado que no se especificó un análisis geográfico, y para simplificar el modelo, se decidió eliminarla. Es poco probable que el código postal individual del propietario sea un predictor directo y robusto del precio de un automóvil de segunda mano en comparación con otras características más directas del vehículo.

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 318962 entries, 0 to 354368
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        318962 non-null  object
 1   Price              318962 non-null  int64 
 2   VehicleType        318962 non-null  object
 3   RegistrationYear   318962 non-null  int64 
 4   Gearbox            318962 non-null  object
 5   Power              318962 non-null  int64 
 6   Model              318962 non-null  object
 7   Mileage            318962 non-null  int64 
 8   RegistrationMonth  318962 non-null  int64 
 9   FuelType           318962 non-null  object
 10  Brand              318962 non-null  object
 11  NotRepaired        318962 non-null  object
 12  DateCreated        318962 non-null  object
 13  LastSeen           318962 non-null  object
dtypes: int64(5), object(9)
memory usage: 36.5+ MB


In [22]:
def clasificar_col_object(df, max_dummy_unique=2, date_threshold=0.8):
    results = {}

    for col in df.select_dtypes(include='object').columns:
        series = df[col].dropna()

        if series.empty:
            results[col] = 'categorical'
            continue

        parsed1 = pd.to_datetime(series, errors='coerce', dayfirst=False)
        parsed2 = pd.to_datetime(series, errors='coerce', dayfirst=True)

        success_ratio = max(
            parsed1.notna().mean(),
            parsed2.notna().mean()
        )

        if success_ratio >= date_threshold:
            results[col] = 'date'
            continue

        if series.nunique() <= max_dummy_unique:
            results[col] = 'dummy'
            continue

        results[col] = 'categorical'

    return results

In [23]:
clasificar_col_object(df, 3)

/tmp/ipython-input-2813464181.py:11: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  parsed1 = pd.to_datetime(series, errors='coerce', dayfirst=False)
/tmp/ipython-input-2813464181.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed1 = pd.to_datetime(series, errors='coerce', dayfirst=False)
/tmp/ipython-input-2813464181.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed2 = pd.to_datetime(series, errors='coerce', dayfirst=True)
/tmp/ipython-input-2813464181.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To e

{'DateCrawled': 'date',
 'VehicleType': 'categorical',
 'Gearbox': 'dummy',
 'Model': 'categorical',
 'FuelType': 'categorical',
 'Brand': 'categorical',
 'NotRepaired': 'dummy',
 'DateCreated': 'date',
 'LastSeen': 'date'}

Los resultados de la función `clasificar_col_object(df, 3)` nos permiten entender el tipo de datos contenido en las columnas de tipo 'object', especialmente útil para la preparación de datos para modelos de Machine Learning. La función clasifica estas columnas en 'date', 'dummy' o 'categorical' basándose en su contenido y el número de valores únicos:

-   **'date'**: Se asigna a columnas que pueden ser convertidas en gran medida a objetos de fecha y hora. Esto es crucial para extraer características temporales.
    *   `DateCrawled`: Contiene la fecha de descarga del perfil.
    *   `DateCreated`: Contiene la fecha de creación del perfil.
    *   `LastSeen`: Contiene la fecha de la última actividad del usuario.

-   **'dummy'**: Se refiere a columnas categóricas con un número muy limitado de valores únicos (en este caso, 2 o 3, ya que se usó `max_dummy_unique=3`). Estas columnas son ideales para ser transformadas mediante One-Hot Encoding (OHE) directamente en variables binarias o de pocas categorías.
    *   `Gearbox`: Contiene 'manual' o 'auto', más el 'unknown' que se agregó.
    *   `NotRepaired`: Contiene 'no', 'yes' y 'unknown'.

-   **'categorical'**: Se asigna a columnas que tienen un número significativo de valores únicos, pero no son fechas. Estas columnas suelen requerir técnicas de codificación categórica más avanzadas o específicas para cada modelo (por ejemplo, OHE para modelos lineales, Label Encoding o Target Encoding para árboles, o ser manejadas directamente por modelos como CatBoost o LightGBM que soportan categorías).
    *   `VehicleType`: Posee varios tipos de vehículos.
    *   `Model`: Tiene una gran cantidad de modelos de coches diferentes.
    *   `FuelType`: Incluye varios tipos de combustible.
    *   `Brand`: Contiene muchas marcas de vehículos.

Esta clasificación es un paso importante para determinar las estrategias de preprocesamiento adecuadas para cada columna antes del entrenamiento del modelo.

In [24]:
df['DateCrawled'] = pd.to_datetime(df['DateCrawled'], format='%d/%m/%Y %H:%M')
df['DateCreated'] = pd.to_datetime(df['DateCreated'], format='%d/%m/%Y %H:%M')
df['LastSeen'] = pd.to_datetime(df['LastSeen'], format='%d/%m/%Y %H:%M')

In [25]:
encoder = OneHotEncoder(drop='first', sparse_output=False)

In [26]:
dummy_cols = ['Gearbox', 'NotRepaired']

encoded_features = encoder.fit_transform(df[dummy_cols])

encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(dummy_cols), index=df.index)

df = pd.concat([df.drop(columns=dummy_cols), encoded_df], axis=1)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 318962 entries, 0 to 354368
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   DateCrawled          318962 non-null  datetime64[ns]
 1   Price                318962 non-null  int64         
 2   VehicleType          318962 non-null  object        
 3   RegistrationYear     318962 non-null  int64         
 4   Power                318962 non-null  int64         
 5   Model                318962 non-null  object        
 6   Mileage              318962 non-null  int64         
 7   RegistrationMonth    318962 non-null  int64         
 8   FuelType             318962 non-null  object        
 9   Brand                318962 non-null  object        
 10  DateCreated          318962 non-null  datetime64[ns]
 11  LastSeen             318962 non-null  datetime64[ns]
 12  Gearbox_manual       318962 non-null  float64       
 13  NotRepaired_unknown

## Entrenamiento del modelo

## Análisis del modelo

# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter

- [x]  Jupyter Notebook está abierto
- [ ]  El código no tiene errores
- [ ]  Las celdas con el código han sido colocadas en orden de ejecución
- [ ]  Los datos han sido descargados y preparados
- [ ]  Los modelos han sido entrenados
- [ ]  Se realizó el análisis de velocidad y calidad de los modelos